<a href="https://colab.research.google.com/github/nanafish/ORS/blob/main/%E4%BC%9A%E5%A0%B4%E3%82%B9%E3%82%B1%E3%82%B8%E3%83%A5%E3%83%BC%E3%83%AB%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# 43期 四半期表 × 地域別会場回数 から
# 週スロット厳守のスケジュール案A/B/C を作るスクリプト
# --------------------------------------------
# 必要ファイル：
#   - 43期四半期表2025-12-01.xlsx
#   - 43期地域別会場回数.xlsx
# ともに Colab 上にアップロードしてから実行
# ============================================

import pandas as pd, re
from collections import defaultdict
from copy import deepcopy

# ==== ファイル名（必要に応じて変更） ====
Q_PATH = "43期四半期表2025-12-01.xlsx"
C_PATH = "43期地域別会場回数.xlsx"

# ==== 読み込み ====
q43 = pd.read_excel(Q_PATH, sheet_name="43期　マスタ ")
q42 = pd.read_excel(Q_PATH, sheet_name="42期　マスタ")
counts_raw = pd.read_excel(C_PATH, sheet_name="AJ43期エリア計画")

# ==== 地域別会場回数 → counts_city ====
counts2 = counts_raw.copy()
counts2["AJ回数"] = pd.to_numeric(counts2.iloc[:, 2], errors="coerce")
counts_city = counts2[~counts2.iloc[:,1].isna() & counts2["AJ回数"].notna()].copy()
counts_city.rename(columns={counts_city.columns[0]:"都道府県列",
                            counts_city.columns[1]:"市"}, inplace=True)

def extract_pref_name(s):
    if not isinstance(s,str): return None
    s = s.strip()
    if not s: return None
    m = re.match(r"([^\d\s]+)", s)
    return m.group(1) if m else None

counts_city["都道府県名"] = counts_city["都道府県列"].apply(extract_pref_name)
counts_city["都道府県名"] = counts_city["都道府県名"].ffill()
counts_city["市"] = counts_city["市"].astype(str).str.strip()
counts_city["AJ回数"] = counts_city["AJ回数"].astype(int)

# ==== 週リスト（42期マスタから） ====
weeks_col = q42.columns[0]
weeks42_raw = q42[weeks_col].dropna().astype(str).tolist()
weeks = []
for w in weeks42_raw:
    w2 = w.replace("ｗ","w")
    if re.match(r"^\d{1,2}-\d{1,2}w$", w2) and w2 not in weeks:
        weeks.append(w2)
nweeks = len(weeks)
week_index = {w:i for i,w in enumerate(weeks)}

# ==== 43期マスタに都道府県名列を付与 ====
def extract_pref_from_code(s):
    if not isinstance(s,str): return None
    s = s.strip()
    if not s: return None
    return re.sub(r"^\d+", "", s).strip()

if "都道府県" in q43.columns:
    q43["都道府県名"] = q43["都道府県"].astype(str).apply(extract_pref_from_code)
else:
    pref_col = [c for c in q43.columns if "都道府" in str(c)]
    if pref_col:
        q43["都道府県名"] = q43[pref_col[0]].astype(str).apply(extract_pref_from_code)
    else:
        q43["都道府県名"] = None

# ==== 都道府県 → 市リスト ====
pref_to_cities = {}
for _,r in counts_city.iterrows():
    pref = r["都道府県名"]
    city = r["市"]
    pref_to_cities.setdefault(pref, set()).add(city)
pref_names = set(pref_to_cities.keys())
shape_vals = ["東","西","北","九州"]

# ==== 四半期表1行 → (都道府県,市) マッピング ====
def map_row_to_city(row):
    wk = row["週"]
    if wk not in week_index:            return None
    if row["形態"] not in shape_vals:   return None

    pref_name = row["都道府県名"]
    area = row["エリア"] if isinstance(row["エリア"], str) else None

    if (not isinstance(pref_name,str) or pref_name=="nan") and \
       isinstance(area,str) and area in pref_names:
        pref_name = area
    if not isinstance(pref_name,str) or pref_name=="nan":
        return None

    cities = pref_to_cities.get(pref_name, set())
    if not cities:
        return None

    # 1) 完全一致
    if area in cities:
        return (pref_name, area)

    # 2) 部分一致（加古川 vs 明石・加古川 等）
    if isinstance(area,str):
        cand = [c for c in cities if (area in c or c in area)]
        if len(cand) == 1:
            return (pref_name, cand[0])

    # 3) 愛媛の特例
    if pref_name=="愛媛" and area==pref_name:
        text = row["会場"] if isinstance(row["会場"], str) else ""
        if ("しこちゅ" in text or "四国中央" in text) and "今治" in cities:
            return (pref_name, "今治")
        if "松山" in cities:
            return (pref_name, "松山")

    # 4) エリア=都道府県名 かつ 1市だけ
    if area == pref_name and len(cities) == 1:
        return (pref_name, list(cities)[0])

    return None

# ==== locked（既存C列埋まり行） ====
locked_events = []
locked_weeks_by_city = defaultdict(list)
week_locked_count = defaultdict(int)

for idx,row in q43.iterrows():
    if not isinstance(row["週"], str) or row["週"] not in week_index:
        continue
    if row["形態"] not in shape_vals:
        continue
    if not isinstance(row["エリア"], str) or row["エリア"].strip()=="":
        continue

    mapped = map_row_to_city(row)
    if not mapped:
        continue
    pref, city = mapped
    cid  = f"{pref}_{city}"
    widx = week_index[row["週"]]

    locked_events.append({"row_index":idx, "week_idx":widx,
                          "week":row["週"], "pref":pref, "city":city})
    locked_weeks_by_city[cid].append(widx)
    week_locked_count[widx] += 1

for cid in locked_weeks_by_city:
    locked_weeks_by_city[cid] = sorted(set(locked_weeks_by_city[cid]))

# ==== 週ごとのベーススロット（行数） ====
base_slots = {i:0 for i in range(nweeks)}
for _,row in q43.iterrows():
    if not isinstance(row["週"], str) or row["週"] not in week_index:
        continue
    if row["形態"] not in shape_vals:
        continue
    widx = week_index[row["週"]]
    base_slots[widx] += 1

# ==== 追加スロット29＋3 ====
extra_slots = {i:0 for i in range(nweeks)}
for w,idx in week_index.items():
    m = int(w.split("-")[0])
    if m in {7,8,10,11,1,2}:
        extra_slots[idx] += 1
for sp in ["9-2w","12-2w","3-2w"]:
    if sp in week_index:
        extra_slots[week_index[sp]] += 1

week_capacity = {i: base_slots[i] + extra_slots[i] for i in range(nweeks)}

# ==== city_meta / locked cap ====
city_meta = {}
for _,r in counts_city.iterrows():
    pref = r["都道府県名"]
    city = r["市"]
    K    = int(r["AJ回数"])
    cid  = f"{pref}_{city}"
    city_meta[cid] = (pref, city, K)

locked_weeks_capped = {}
for cid,weeks_list in locked_weeks_by_city.items():
    if cid not in city_meta:
        continue
    pref, city, K = city_meta[cid]
    locked_weeks_capped[cid] = sorted(weeks_list)[:K]

# ==== 均等配置用ヘルパー ====
def assign_weeks_even(k, pattern_idx, nweeks):
    if k <= 0:
        return []
    base = [round(nweeks * i / k) for i in range(k)]
    base = [min(max(b,0), nweeks-1) for b in base]
    offset = int(round(nweeks * pattern_idx / (3 * max(1,k))))
    pos = sorted(((b + offset) % nweeks for b in base))
    used=set(); res=[]
    for p in pos:
        while p in used:
            p = (p+1) % nweeks
        used.add(p); res.append(p)
    return sorted(res)

def build_events_for_pattern(pattern_idx):
    events=[]
    for cid,(pref,city,K) in city_meta.items():
        locked_cap = locked_weeks_capped.get(cid, [])
        L_eff = min(len(locked_cap), K)
        remain = max(K - L_eff, 0)
        if remain <= 0:
            continue
        base = assign_weeks_even(K, pattern_idx, nweeks)
        base2 = base.copy()
        for lw in sorted(set(locked_cap))[:L_eff]:
            if not base2: break
            dists = [abs(lw-b) for b in base2]
            j = min(range(len(base2)), key=lambda i: dists[i])
            base2.pop(j)
        for pos in base2[:remain]:
            events.append({"cid":cid, "pref":pref, "city":city,
                           "target_idx":pos})
    return events

def schedule_events(events):
    week_used = week_locked_count.copy()
    city_weeks = {cid:set(locked_weeks_by_city.get(cid, []))
                  for cid in city_meta.keys()}
    slot_assign = []
    unassigned  = []
    max_shift = nweeks
    for e in sorted(events, key=lambda x: x["target_idx"]):
        cid = e["cid"]; t = e["target_idx"]
        assigned = False
        for shift in range(0, max_shift+1):
            if assigned: break
            if shift == 0:
                candidates = [t]
            else:
                candidates = [(t+shift)%nweeks, (t-shift)%nweeks]
            for widx in candidates:
                if (week_used.get(widx,0) < week_capacity[widx] and
                    widx not in city_weeks[cid]):
                    week_used[widx] = week_used.get(widx,0) + 1
                    city_weeks[cid].add(widx)
                    slot_assign.append({"cid":cid,"pref":e["pref"],
                                        "city":e["city"],"week_idx":widx})
                    assigned = True
                    break
        if not assigned:
            unassigned.append(e)
    return slot_assign, week_used, city_weeks, unassigned

def build_schedule(label, assign):
    rows=[]
    for ev in locked_events:
        rows.append({"案":label,"週":ev["week"],
                     "都道府県名":ev["pref"],
                     "市":ev["city"],
                     "source":"locked"})
    for ev in assign:
        rows.append({"案":label,"週":weeks[ev["week_idx"]],
                     "都道府県名":ev["pref"],
                     "市":ev["city"],
                     "source":"assigned"})
    df = pd.DataFrame(rows)
    df = df.sort_values(["都道府県名","市","週"])
    df["回数番号"] = df.groupby(["都道府県名","市"]).cumcount()+1
    df = df.merge(counts_city[["都道府県名","市","AJ回数"]],
                  on=["都道府県名","市"], how="left")
    df = df.rename(columns={"AJ回数":"AJ回数計画"})
    final_counts = df.groupby(["都道府県名","市"]).size().reset_index(name="最終回数")
    df = df.merge(final_counts, on=["都道府県名","市"], how="left")
    df["week_idx"] = df["週"].map(week_index)
    df = df.sort_values(["week_idx","都道府県名","市","回数番号"])
    return df[["案","週","都道府県名","市","回数番号","AJ回数計画","最終回数","source"]]

# ==== 案A/B/Cを作成 ====
eventsA = build_events_for_pattern(0)
assignA, week_usedA, city_weeksA, unassignedA = schedule_events(eventsA)
dfA = build_schedule("案A", assignA)

eventsB = build_events_for_pattern(1)
assignB, week_usedB, city_weeksB, unassignedB = schedule_events(eventsB)
dfB = build_schedule("案B", assignB)

eventsC = build_events_for_pattern(2)
assignC, week_usedC, city_weeksC, unassignedC = schedule_events(eventsC)
dfC = build_schedule("案C", assignC)

# ==== CSV出力 ====
dfA.to_csv("43期AJスケジュール案A_週スロット厳守_完全版.csv", index=False, encoding="utf-8-sig")
dfB.to_csv("43期AJスケジュール案B_週スロット厳守_完全版.csv", index=False, encoding="utf-8-sig")
dfC.to_csv("43期AJスケジュール案C_週スロット厳守_完全版.csv", index=False, encoding="utf-8-sig")

print("A/B/C 出力完了")
